In [1]:
def trans_mat(F, k):
    MS = MatrixSpace(F, k, k);

    # Want k x k matrices
    # T is identity except swapping 1st and 2nd positions
    T = MS([[1 if j == i else 0 for j in range(k)] if i > 1 else [abs(j - i) if j < 2 else 0 for j in range(k)] for i in range(k)])
    return T

def cyc_mat(F, k):
    F = QQ;
    MS = MatrixSpace(F, k, k);

    # Want k x k matrices
    # T is identity except swapping 1st and 2nd positions
    C = MS([[1 if j == i - 1 else 0 for j in range(k)] if i > 0 else [1 if j == k - 1 else 0 for j in range(k)] for i in range(k)])
    return C

def zero_mat(F, k):
    MS = MatrixSpace(F, k, k);

    # Want k x k matrices
    # T is identity except swapping 1st and 2nd positions
    O = MS(0)
    return O

def get_gen_mats(F, l, k):
    T = trans_mat(F, k);
    C = cyc_mat(F, k);
    I = matrix.identity(k)
    O = zero_mat(F, k);
    
    # Permute the blocks themselves
    # Permute the elements within the blocks (using tranpose and cyclic matrices from above)
    gens = []
    
    # Matrices for permuting the blocks among themselves, S_l
    # One block transpose matrix, one cycle.
    BT = block_matrix([[I if j == i else O for j in range(l)] if i > 1 else [abs(j - i) * I if j < 2 else O for j in range(l)] for i in range(l)])
    BC = block_matrix([[I if j == i - 1 else O for j in range(l)] if i > 0 else [I if j == l - 1 else O for j in range(l)] for i in range(l)])
    gens.extend([BT, BC])
    
    # Matrices for permutations within the blocks, S_k.
    # One tranposition and one cycle for each block.
    for b in range(l):
        T_b = block_matrix([[(T if i == b else I) if j == i else O for j in range(l)] for i in range(l)])
        C_b = block_matrix([[(C if i == b else I) if j == i else O for j in range(l)] for i in range(l)])
        gens.extend([T_b, C_b])

    return gens

In [2]:
def get_inv_gens(F, l, k):
    gen_mats = get_gen_mats(F, l, k)
    G = MatrixGroup(gen_mats);
    gens = G.invariant_generators()
    return gens

In [14]:
def get_inds(F, l, k, inv_gens):
    n_gens = len(inv_gens)
    str_x = ['x%d' %(i + 1) for i in range(l * k)]
    str_y = ['y%d' %(i + 1) for i in range(n_gens)]
    str_inds = str_x + str_y
    inds = QQ[', '.join(str_inds)].gens()
    return inds

In [16]:
def get_moment(n, F, l, k, inv_gens):
    inds = get_inds(F, l, k, inv_gens)
    # Note that x comes before y in inds, so this just returns a partition of x.
    x = [inds[j * k: (j + 1) * k] for j in range(l)]
    P = 0
    
    U = Tuples(range(k), l).list()
    for u in U:
        t = 1
        for j in range(l):
            u_j = u[j]
            t *= x[j][u_j]
        P += t^n
    return P

In [30]:
def get_gen_expr(f, F, inv_gens, l, k):
    inds = get_inds(F, l, k, inv_gens)
    n_inds = len(inds)
    n_gens = len(inv_gens)
    y = [inds[i] for i in range(n_inds - n_gens, n_inds)]
    
    J_gens = [inv_gens[i] - y[i] for i in range(n_gens)]
    J = ideal(J_gens)
    expr = J.reduce(f)
    return expr

Get the generators for the $l = 2, k = 2$ case. Write the first through third moments as polynomials in the invariants.

In [22]:
F, l, k = QQ, 2, 2
inv_gens = get_inv_gens(F, l, k)
inv_gens

[x1 + x2 + x3 + x4,
 x1^2 + x2^2 + x3^2 + x4^2,
 x1*x2 + x3*x4,
 x1^3 + x2^3 + x3^3 + x4^3,
 x1^4 + x2^4 + x3^4 + x4^4]

In [23]:
P_1 = get_moment(1, F, l, k, inv_gens)
P_1

x1*x3 + x2*x3 + x1*x4 + x2*x4

In [24]:
P_2 = get_moment(2, F, l, k, inv_gens)
P_2

x1^2*x3^2 + x2^2*x3^2 + x1^2*x4^2 + x2^2*x4^2

In [26]:
P_3 = get_moment(3, F, l, k, inv_gens)
P_3

x1^3*x3^3 + x2^3*x3^3 + x1^3*x4^3 + x2^3*x4^3

In [36]:
f_1 = get_gen_expr(P_1, F, inv_gens, l, k)
print('moment 1: ' + str(f_1))

f_2 = get_gen_expr(P_2, F, inv_gens, l, k)
print('moment 2: ' + str(f_2))

f_3 = get_gen_expr(P_3, F, inv_gens, l, k)
print('moment 3: ' + str(f_3))

moment 1: 1/2*y1^2 - 1/2*y2 - y3
moment 2: 1/12*y1^4 - 1/2*y1^2*y2 + 3/4*y2^2 - y3^2 + 2/3*y1*y4 - y5
moment 3: 1/16*y1^4*y2 + 1/8*y1^4*y3 - 3/8*y1^2*y2^2 - 3/2*y1^2*y2*y3 + 3/4*y1^2*y3^2 + 7/16*y2^3 + 9/8*y2^2*y3 - 3/4*y2*y3^2 - 5/2*y3^3 + 1/2*y1*y2*y4 + 5/2*y1*y3*y4 + 1/2*y4^2 - 9/8*y2*y5 - 9/4*y3*y5


Get the generators for the $l = 2, k = 3$ case. Write the first through third moments as polynomials in the invariants.

In [37]:
F, l, k = QQ, 2, 3
inv_gens = get_inv_gens(F, l, k)
inv_gens

[x1 + x2 + x3 + x4 + x5 + x6,
 x1^2 + x2^2 + x3^2 + x4^2 + x5^2 + x6^2,
 x1*x2 + x1*x3 + x2*x3 + x4*x5 + x4*x6 + x5*x6,
 x1^3 + x2^3 + x3^3 + x4^3 + x5^3 + x6^3,
 x1^2*x2 + x1*x2^2 + x1^2*x3 + x2^2*x3 + x1*x3^2 + x2*x3^2 + x4^2*x5 + x4*x5^2 + x4^2*x6 + x5^2*x6 + x4*x6^2 + x5*x6^2,
 x1^4 + x2^4 + x3^4 + x4^4 + x5^4 + x6^4,
 x1^3*x2 + x1*x2^3 + x1^3*x3 + x2^3*x3 + x1*x3^3 + x2*x3^3 + x4^3*x5 + x4*x5^3 + x4^3*x6 + x5^3*x6 + x4*x6^3 + x5*x6^3,
 x1^5 + x2^5 + x3^5 + x4^5 + x5^5 + x6^5,
 x1^6 + x2^6 + x3^6 + x4^6 + x5^6 + x6^6]

In [38]:
P_1 = get_moment(1, F, l, k, inv_gens)
print(P_1)
P_2 = get_moment(2, F, l, k, inv_gens)
print(P_2)
P_3 = get_moment(3, F, l, k, inv_gens)
print(P_3)

x1*x4 + x2*x4 + x3*x4 + x1*x5 + x2*x5 + x3*x5 + x1*x6 + x2*x6 + x3*x6
x1^2*x4^2 + x2^2*x4^2 + x3^2*x4^2 + x1^2*x5^2 + x2^2*x5^2 + x3^2*x5^2 + x1^2*x6^2 + x2^2*x6^2 + x3^2*x6^2
x1^3*x4^3 + x2^3*x4^3 + x3^3*x4^3 + x1^3*x5^3 + x2^3*x5^3 + x3^3*x5^3 + x1^3*x6^3 + x2^3*x6^3 + x3^3*x6^3


In [39]:
f_1 = get_gen_expr(P_1, F, inv_gens, l, k)
print('moment 1: ' + str(f_1))

f_2 = get_gen_expr(P_2, F, inv_gens, l, k)
print('moment 2: ' + str(f_2))

f_3 = get_gen_expr(P_3, F, inv_gens, l, k)
print('moment 3: ' + str(f_3))

moment 1: 1/2*y1^2 - 1/2*y2 - y3
moment 2: -1/12*y1^4 + 2/3*y1^2*y2 + 1/3*y1^2*y3 + 1/12*y2^2 - 2/3*y2*y3 + 1/3*y3^2 - y1*y4 - y1*y5 + 1/3*y6 + 4/3*y7
moment 3: -3/40*y1^4*y3 + 3/20*y1^2*y2^2 + 9/20*y1^2*y2*y3 + 3/10*y1^2*y3^2 + 1/20*y1^3*y4 - 3/40*y1^3*y5 - 1/10*y2^3 - 9/40*y2^2*y3 - 3/5*y2*y3^2 + 2/5*y3^3 - 13/20*y1*y2*y4 - 7/10*y1*y3*y4 + 9/40*y1*y2*y5 - 9/20*y1*y3*y5 - 7/20*y1^2*y6 + 1/10*y1^2*y7 + 4/5*y4^2 - 3/20*y4*y5 - 9/20*y5^2 + 7/10*y2*y6 + 13/20*y3*y6 - 1/5*y2*y7 + 11/10*y3*y7 + 9/10*y1*y8 - 3/2*y9


Get invariant generators for all (l, k) where l, k <= 4. Also get expressions for the first 10 moments in terms of those invariant generators.

In [55]:
F = QQ
max_l, max_k = 4, 4
for l in range(2, max_l + 1):
    for k in range(2, max_k + 1):
        print("l = {people}, k = {types}".format(people = l, types = k))
        inv_gens = get_inv_gens(F, l, k)
        print(str(inv_gens))
        for n in range(1, 5):
            mu = get_moment(n, F, l, k, inv_gens)
            expr = get_gen_expr(mu, F, inv_gens, l, k)
            print("mu_{order} = {poly}".format(order = n, poly = expr))
        if l != max_l or k != max_k:
            print('\n')

l = 2, k = 2
[x1 + x2 + x3 + x4, x1^2 + x2^2 + x3^2 + x4^2, x1*x2 + x3*x4, x1^3 + x2^3 + x3^3 + x4^3, x1^4 + x2^4 + x3^4 + x4^4]
mu_1 = 1/2*y1^2 - 1/2*y2 - y3
mu_2 = 1/12*y1^4 - 1/2*y1^2*y2 + 3/4*y2^2 - y3^2 + 2/3*y1*y4 - y5
mu_3 = 1/16*y1^4*y2 + 1/8*y1^4*y3 - 3/8*y1^2*y2^2 - 3/2*y1^2*y2*y3 + 3/4*y1^2*y3^2 + 7/16*y2^3 + 9/8*y2^2*y3 - 3/4*y2*y3^2 - 5/2*y3^3 + 1/2*y1*y2*y4 + 5/2*y1*y3*y4 + 1/2*y4^2 - 9/8*y2*y5 - 9/4*y3*y5
mu_4 = 1/12*y1^4*y2^2 + 1/6*y1^4*y3^2 - 1/2*y1^2*y2^3 - y1^2*y2^2*y3 + 1/2*y2^4 + y2^3*y3 - 1/2*y2^2*y3^2 - 2*y2*y3^3 - y3^4 + 2/3*y1*y2^2*y4 + 2*y1*y2*y3*y4 + 4/3*y1*y3^2*y4 - y2^2*y5 - 2*y2*y3*y5 - y3^2*y5 + 1/4*y5^2


l = 2, k = 3
   skipping text from `parameter`// ** redefining i_par (parameter def i_par; parameter list #;  ) standard.lib::groebner:840

[x1 + x2 + x3 + x4 + x5 + x6, x1^2 + x2^2 + x3^2 + x4^2 + x5^2 + x6^2, x1*x2 + x1*x3 + x2*x3 + x4*x5 + x4*x6 + x5*x6, x1^3 + x2^3 + x3^3 + x4^3 + x5^3 + x6^3, x1^2*x2 + x1*x2^2 + x1^2*x3 + x2^2*x3 + x1*x3^2 + x2*x3^

KeyboardInterrupt: 

In [61]:
F = QQ
max_l, max_k = 3, 3
for l in range(2, max_l + 1):
    for k in range(2, max_k + 1):
        print("l = {people}, k = {types}".format(people = l, types = k))
        inv_gens = get_inv_gens(F, l, k)
        print(str(inv_gens))
        for n in range(1, 4):
            mu = get_moment(n, F, l, k, inv_gens)
            expr = get_gen_expr(mu, F, inv_gens, l, k)
            print("mu_{order} = {poly}".format(order = n, poly = expr))
        if l != max_l or k != max_k:
            print('\n')

l = 2, k = 2
[x1 + x2 + x3 + x4, x1^2 + x2^2 + x3^2 + x4^2, x1*x2 + x3*x4, x1^3 + x2^3 + x3^3 + x4^3, x1^4 + x2^4 + x3^4 + x4^4]
mu_1 = 1/2*y1^2 - 1/2*y2 - y3   skipping text from `parameter`
mu_2 = 1/12*y1^4 - 1/2*y1^2*y2 + 3/4*y2^2 - y3^2 + 2/3*y1*y4 - y5
mu_3 = 1/16*y1^4*y2 + 1/8*y1^4*y3 - 3/8*y1^2*y2^2 - 3/2*y1^2*y2*y3 + 3/4*y1^2*y3^2 + 7/16*y2^3 + 9/8*y2^2*y3 - 3/4*y2*y3^2 - 5/2*y3^3 + 1/2*y1*y2*y4 + 5/2*y1*y3*y4 + 1/2*y4^2 - 9/8*y2*y5 - 9/4*y3*y5


l = 2, k = 3
// ** redefining i_par (parameter def i_par; parameter list #;  ) standard.lib::groebner:840

[x1 + x2 + x3 + x4 + x5 + x6, x1^2 + x2^2 + x3^2 + x4^2 + x5^2 + x6^2, x1*x2 + x1*x3 + x2*x3 + x4*x5 + x4*x6 + x5*x6, x1^3 + x2^3 + x3^3 + x4^3 + x5^3 + x6^3, x1^2*x2 + x1*x2^2 + x1^2*x3 + x2^2*x3 + x1*x3^2 + x2*x3^2 + x4^2*x5 + x4*x5^2 + x4^2*x6 + x5^2*x6 + x4*x6^2 + x5*x6^2, x1^4 + x2^4 + x3^4 + x4^4 + x5^4 + x6^4, x1^3*x2 + x1*x2^3 + x1^3*x3 + x2^3*x3 + x1*x3^3 + x2*x3^3 + x4^3*x5 + x4*x5^3 + x4^3*x6 + x5^3*x6 + x4*x6^3 + x5*x6

KeyboardInterrupt: 